## Draw Sankey of UK Energy System

The first step involves importing all the necessary packages to crete the Sankey

In [1]:
import pandas as pd
from sankeyview import *
from sankeyview.jupyter import show_sankey
#from trace_sankey_helpers import show_sample, show_variance, flows_from_trace #, show_sankey

Then we proceed to load the data that we want to draw. In one worksheet we have all the flows, while in the other worksheet we have information about each process in the diagram.

In [3]:
FILENAME = './UK_Energy_Sankey_data.xlsx'
processes = pd.read_excel(FILENAME, sheetname='Processes', index_col=0)
flows = pd.read_excel(FILENAME, sheetname='Flows')


We need to make sure that there is no repetition of flows in the data. Therefore we group and sum all flows.

In [4]:
flows = flows.groupby(['source', 'target', 'type'], as_index=False).sum()

We now create the variable which will be used to plot the diagram

In [17]:
dataset = Dataset(flows, dim_process=processes)

In [18]:
source_grouping = Partition.Simple(
    'process',
    ['Gas',
        'Liquid fuel',
    'Coal',
 'Nuclear',
 'Solid biomass & Waste',
 'Renewable',
'Primary Electricity',
          'Heat',
     'Electricity'
])

sector_grouping = Partition.Simple(
    'process',
    ['Road','Rail','Navigation','Aviation',
'Industry',
'Commercial and public services',
'Residential',
'Agriculture/forestry'
])

EndUse_grouping =Partition.Simple(
    'process',
    ['Mechanical',
    'Process Heat-Direct',
     'Process Heat-Boiler',
     'Space Heat',
     'Hot Water',
     
     'Space Cooling',
     'Process Cooling',
     'Light',
     'Information'
])

device_grouping1 = Partition.Simple(
    'process',
    ['Boiler','Burner','Electric Heater','Spark Ignition Engine',
     'Diesel Engine',
     'Gas Turbine',
     'Electric Motor',    
    'Cooler',
     'Light Device',
     'Electronics'
    ])
useful_grouping = Partition.Simple(
    'process', ['Heating',
                            'Motion',

        'Cool',

'Illumination',
'Information',
        'Loss'
    ])
trans_grouping = Partition.Simple(
    'process', ['Transformation','Heat Plant','Power Plant'])

In [19]:
# I should make this a two layer diagram... but I will have to ask rick the synthax for it
# Also, i probably need to learn how to use his adjustment software.
default_grouping =  Partition.Simple('process', processes.index.values)

nodes = {
   # 'Ren':        Node(),
   # 'a':          Node(),
   # 'Biomass & Waste':           Node(selection="Level == 10" ),
    'inputs':           ProcessGroup(selection="Level == 1", partition=source_grouping),
     'Power Plants':           ProcessGroup(selection="Level == 11",partition=trans_grouping ),
    #'Sector':           ProcessGroup(selection='Level == 3', partition=device_grouping),
    #'Transport':           ProcessGroup(selection='Level == 31'),    
   #'End-use':          ProcessGroup(selection='Level == 4', partition=EndUse_grouping),
    'Devices':      ProcessGroup(selection='Level == 2', partition=device_grouping1),
    'Useful':      ProcessGroup(selection='Level == 3', partition=useful_grouping)

   # 'loss':      Node(selection='Level == 21', grouping=device_grouping), 
    
}
order = [
    ['inputs'],
    ['Power Plants'],
    ['Devices'],
    ['Useful']
]
bundles = [
    Bundle('inputs', 'Devices'),
    Bundle('inputs', 'Power Plants'),
    Bundle('Power Plants', 'Devices'),
    Bundle('Power Plants', 'Useful'),
    Bundle('Devices','Useful')
]

vd = SankeyDefinition(nodes, bundles, order,flow_partition= dataset.partition('type'))

In [20]:
#from palettable import qualitative
#palette = qualitative.Pastel1_8.hex_colors
from collections import defaultdict
palette = defaultdict(lambda: '#999')
palette['Coal'] = 'black'
palette['Liquid fuel'] = 'red'
palette['Gas'] = 'blue'
palette['Solid biomass & Waste'] = 'Green'
palette['Primary Electricity'] = 'Green'
palette['Biogas'] = 'Green'
palette['Renewable'] = 'Green'
palette['Nuclear'] = 'darkviolet'
palette['Wind'] = 'aquamarine'
palette['Hydro'] = 'aquamarine'
palette['Solar PV'] = 'aquamarine'
palette['Waste'] = 'black'
palette['Electricity'] = 'yellow'
palette['Heat'] = 'orange'
palette['Loss'] = 'gray'
palette['Renewable Heat'] = 'orange'
palette['Fuel'] = 'black'


In [21]:
show_sankey(vd, dataset, width=900, height=600,palette=palette).auto_save_svg('Uk_Sankey_Template.svg')